# Import

In [1]:
# ==============================================================================
# H2: 1. Imports and Setup
# ==============================================================================
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

print(f"PyTorch Version: {torch.__version__}")
print(f"Torchvision Version: {torchvision.__version__}")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")


PyTorch Version: 2.6.0+cu124
Torchvision Version: 0.21.0+cu124
Using device: cuda


# Attention Blocks

## SE Block

In [2]:
# H3: 2.1 Squeeze-and-Excitation (SE) Block
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        # Squeeze: Global Average Pooling layer
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        # Excitation: A two-layer MLP
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        # Squeeze
        y = self.avg_pool(x).view(b, c)
        # Excitation
        y = self.fc(y).view(b, c, 1, 1)
        # Scale the original input
        return x * y.expand_as(x)

## Channel Attention

In [3]:
# H3: 2.2 Convolutional Block Attention Module (CBAM)
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

## Spatial Attention

In [4]:
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

## CBAM

In [5]:
class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x

# CNN Architecture

## Basic CNN Block

In [6]:
# H3: 3.1 Basic Building Block
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, attention_module=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)

        # Optional attention module
        self.attention = attention_module

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        if self.attention is not None:
            out = self.attention(out)
            
        out += identity
        out = self.relu(out)
        return out

## Main CNN

In [7]:
# H3: 3.2 Main CNN Model
class CNNWithAttention(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, in_channels=3, attention_type='none'):
        super(CNNWithAttention, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, attention_type=attention_type)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2, attention_type=attention_type)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2, attention_type=attention_type)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, attention_type=attention_type)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride, attention_type):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            attention_module = None
            if attention_type == 'se':
                attention_module = SELayer(planes)
            elif attention_type == 'cbam':
                attention_module = CBAM(planes)
            
            layers.append(block(self.in_planes, planes, stride, attention_module))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# Training-Testing Code Definition

## Training Definition

In [8]:
# H3: 4.1 Training Function 
def train_one_epoch(model, loader, optimizer, loss_fn, device, scaler): # Add scaler as an argument
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    
    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        # The core AMP change is here
        with torch.amp.autocast('cuda'):
            outputs = model(images)
            loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        
        # Scaler handles the backward pass and scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    train_accuracy = 100 * correct / total
    return total_loss / len(loader), train_accuracy

## Testing Definition

In [9]:
# H3: 4.2 Evaluation Function (for final testing)
def evaluate_final(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Final Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

# Data Loading

In [10]:
# ==============================================================================
# H2: 5. Dataset Loading Helper --- REPLACE THIS ENTIRE BLOCK ---
# ==============================================================================
def get_dataset(name, root='./data'):
    # Transform for small 32x32 images like CIFAR
    transform_32 = transforms.Compose([
        transforms.ToTensor(),
        # Using standard CIFAR-10 normalization stats as a good default for 32x32
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    # Transform for larger images, resizing to a standard 224x224
    transform_224 = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # Using ImageNet normalization stats, a common default for larger images
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Specific transform for EuroSAT which has different channel statistics
    transform_eurosat = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.3444, 0.3803, 0.4078], std=[0.2034, 0.1367, 0.1148])
    ])

    if name.lower() == 'cifar10':
        train_ds = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_32)
        test_ds = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_32)
        return train_ds, test_ds, 10, 3

    elif name.lower() == 'cifar100':
        train_ds = torchvision.datasets.CIFAR100(root=root, train=True, download=True, transform=transform_32)
        test_ds = torchvision.datasets.CIFAR100(root=root, train=False, download=True, transform=transform_32)
        return train_ds, test_ds, 100, 3
    
    elif name.lower() == 'oxford102flowers':
        train_ds = torchvision.datasets.Flowers102(root=root, split='train', download=True, transform=transform_224)
        test_ds = torchvision.datasets.Flowers102(root=root, split='test', download=True, transform=transform_224)
        return train_ds, test_ds, 102, 3
        
    elif name.lower() == 'eurosat':
        dataset = torchvision.datasets.EuroSAT(root=root, download=True, transform=transform_eurosat)
        # EuroSAT does not have a predefined split, so we create one (80/20).
        train_size = int(0.8 * len(dataset))
        test_size = len(dataset) - train_size
        train_ds, test_ds = torch.utils.data.random_split(dataset, [train_size, test_size])
        return train_ds, test_ds, 10, 3
        
    else:
        raise ValueError(f"Unknown or un-configured dataset: {name}")

# Cleanup Helper

In [11]:
# ==============================================================================
# H2: [NEW] Memory Cleanup Helper
# ==============================================================================
import gc

def cleanup(model=None, optimizer=None):
    """
    Explicitly deletes model and optimizer and clears GPU cache.
    Helps to prevent CUDA out of memory errors in sequential experiments.
    """
    if optimizer:
        del optimizer
    if model:
        del model
    
    # Perform garbage collection
    gc.collect()
    
    # Empty the CUDA cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# Experiment Runner

In [12]:
# ==============================================================================
# H2: 6. Experiment Runner --- REPLACE THIS ENTIRE BLOCK ---
# ==============================================================================

# H3: 6.1 Experiment Configuration
EPOCHS = 25       
BATCH_SIZE = 32   
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4

datasets_to_run = ['CIFAR10', 'CIFAR100', 'Oxford102Flowers', 'EuroSAT']
attention_types = ['none', 'se', 'cbam']
results = {}



scaler = torch.amp.GradScaler('cuda')



# H3: 6.2 Main Experiment Loop
for dataset_name in datasets_to_run:
    print("\n" + "="*80)
    print(f" H2: Running Experiments on Dataset: {dataset_name.upper()}")
    print("="*80 + "\n")

    try:
        train_ds, test_ds, num_classes, in_channels = get_dataset(dataset_name)
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
        test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    except Exception as e:
        print(f"Could not load dataset {dataset_name}. Skipping. Error: {e}")
        continue
    
    results[dataset_name] = {}

    for attention_type in attention_types:
        print("\n" + "-"*80)
        print(f"H3: Training Model with Attention Type: {attention_type.upper()}")
        print("-"*80)

        model = CNNWithAttention(BasicBlock, [2, 2, 2, 2], num_classes=num_classes, in_channels=in_channels, attention_type=attention_type).to(DEVICE)
        
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

        for epoch in range(EPOCHS):
            # Pass the scaler to the function
            train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, loss_fn, DEVICE, scaler)
            scheduler.step()
            print(f"Epoch {epoch+1}/{EPOCHS} -> Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

        
        final_test_accuracy = evaluate_final(model, test_loader, DEVICE)
        print(f"\nFINAL TEST ACCURACY for {attention_type.upper()} model on {dataset_name}: {final_test_accuracy:.2f}%\n")
        results[dataset_name][attention_type] = final_test_accuracy
        
        # ===> ADD THE CLEANUP STEP HERE <===
        # This will free up GPU memory before the next model is created.
        cleanup(model, optimizer)


 H2: Running Experiments on Dataset: CIFAR10



100%|██████████| 170M/170M [00:29<00:00, 5.78MB/s]



--------------------------------------------------------------------------------
H3: Training Model with Attention Type: NONE
--------------------------------------------------------------------------------


Training: 100%|██████████| 1563/1563 [00:33<00:00, 47.11it/s]


Epoch 1/25 -> Train Loss: 1.3278, Train Accuracy: 51.65%


...

Epoch 24/25 -> Train Loss: 0.0113, Train Accuracy: 99.69%


Training: 100%|██████████| 675/675 [07:33<00:00,  1.49it/s]


Epoch 25/25 -> Train Loss: 0.0107, Train Accuracy: 99.74%


Final Testing: 100%|██████████| 169/169 [00:40<00:00,  4.14it/s]



FINAL TEST ACCURACY for CBAM model on EuroSAT: 97.37%



# Result Summary

In [13]:
# ==============================================================================
# H2: 7. Final Results Summary
# ==============================================================================
print("\n" + "="*80)
print(" H2: Final Experiment Results Summary")
print("="*80 + "\n")

# Header
header = f"{'Dataset':<15}"
for att_type in attention_types:
    header += f" | {att_type.upper():<10}"
print(header)
print("-" * len(header))

# Rows
for dataset_name, att_results in results.items():
    row = f"{dataset_name:<15}"
    for att_type in attention_types:
        acc = att_results.get(att_type, 'N/A')
        row += f" | {acc:.2f}%{'':<5}" if isinstance(acc, float) else f" | {'N/A':<10}"
    print(row)


 H2: Final Experiment Results Summary

Dataset         | NONE       | SE         | CBAM      
------------------------------------------------------
CIFAR10         | 86.97%      | 87.01%      | 85.89%     
CIFAR100        | 57.26%      | 59.28%      | 58.07%     
Oxford102Flowers | 28.13%      | 37.53%      | 38.74%     
EuroSAT         | 97.41%      | 97.61%      | 97.37%     
